In [6]:
from collections import defaultdict
import pandas as pd
from datetime import date
from dateutil.relativedelta import relativedelta

DIAS_MES_PADRAO = 30

def vfuturo(vp, i, n):
    return vp*(1+i)**n

def vpresente(vf, i, n):
    return vf/(1+i)**n

def caculo_fator_de_financiamento(i, n):
    return (1 - (1 + i)**(-n)) / i

def taxa_mensal_para_dia(i):
    return (1 + i) ** (1/DIAS_MES_PADRAO) - 1

# Relação prestação x valor solicitado

$$ PV = PMT * FPV(i,n) $$

com:
 
$$ FPV(i, n) = \frac{1 - (1 + i)^{-n}}{i}$$

Onde:

- PV: Valor solicitado
- PMT: Valor da prestação
- n: Número de prestações
- i: taxa de jutos


## Informando o valor solicitado 

In [ ]:
taxa_mensal = 0.01
valor_solicitado = 20_000
numero_de_prestacoes = 15
data_liberacao = date(2021, 1, 5)
dia_vencimento_parcela = 5
taxa_iof_dia = 0.000082
taxa_iof_comp_mensal = 0.0038

In [8]:
taxa_dia = taxa_mensal_para_dia(taxa_mensal)
taxa_dia

0.00033173270623421125

In [10]:
data_vencimento_parcelas = [
    (data_liberacao + relativedelta(months=prestacao)).replace(day=dia_vencimento_parcela)
    for prestacao in range(1, numero_de_prestacoes+1)
]

data_vencimento_parcelas

[datetime.date(2021, 2, 5),
 datetime.date(2021, 3, 5),
 datetime.date(2021, 4, 5),
 datetime.date(2021, 5, 5),
 datetime.date(2021, 6, 5),
 datetime.date(2021, 7, 5),
 datetime.date(2021, 8, 5),
 datetime.date(2021, 9, 5),
 datetime.date(2021, 10, 5),
 datetime.date(2021, 11, 5),
 datetime.date(2021, 12, 5),
 datetime.date(2022, 1, 5),
 datetime.date(2022, 2, 5),
 datetime.date(2022, 3, 5),
 datetime.date(2022, 4, 5)]

In [11]:
fator_de_financiamento = 0
for parcela in range(1, numero_de_prestacoes+1):
    dias = (data_vencimento_parcelas[parcela - 1] - data_liberacao).days
    fator_de_financiamento += vpresente(1, taxa_dia, dias)

fator_de_financiamento

13.853788441846143

In [12]:
total_dias = (data_vencimento_parcelas[-1] - data_liberacao).days
total_dias

455

In [13]:
fator_de_financiamento_meses = caculo_fator_de_financiamento(taxa_mensal, numero_de_prestacoes)
fator_de_financiamento_dias = caculo_fator_de_financiamento(taxa_dia, total_dias)

print(f"Fator de fianciamento usando meses {fator_de_financiamento_meses}")
print(f"Fator de fianciamento usando dias {fator_de_financiamento_dias}")
print(f"Fator de fianciamento usando recebivel unitario com juros diarios {fator_de_financiamento}")

Fator de fianciamento usando meses 13.865052517162113
Fator de fianciamento usando dias 422.26102754397164
Fator de fianciamento usando recebivel unitario com juros diarios 13.853788441846143


In [14]:
valor_da_prestacao = valor_solicitado / fator_de_financiamento
print(f"{valor_da_prestacao=:10.2f}")

valor_da_prestacao=   1443.65


In [ ]:
saldo_devedor = valor_solicitado

tabela = defaultdict(list)

for parcela in range(1, numero_de_prestacoes+1):

    dia_vencimento_parcela_atual = data_vencimento_parcelas[parcela - 1]
    if parcela == 1:
        dias_parcela = (dia_vencimento_parcela_atual - data_liberacao).days
    else:
        dia_vencimento_parcela_anterior = data_vencimento_parcelas[parcela - 2]
        dias_parcela = (dia_vencimento_parcela_atual - dia_vencimento_parcela_anterior).days
    dias_acumulados = (dia_vencimento_parcela_atual - data_liberacao).days

    juros = vfuturo(saldo_devedor, taxa_dia, dias_parcela) - saldo_devedor
    amortizacao = valor_da_prestacao - juros # principal

    dias_iof = min(365, dias_acumulados)

    taxa_efetiva_iof = (taxa_iof_dia * dias_iof + taxa_iof_comp_mensal)

    iof_parcela = amortizacao * taxa_efetiva_iof

    saldo_devedor -= amortizacao

    tabela['parcela'].append(parcela)
    tabela['data_vencimento'].append(dia_vencimento_parcela_atual)
    tabela['dias_parcela'].append(dias_parcela)
    tabela['dias_acumulados'].append(dias_acumulados)
    tabela['saldo_devedor'].append(saldo_devedor)
    tabela['amortizacao'].append(amortizacao)
    tabela['juros'].append(juros)
    tabela['valor_prestacao'].append(valor_da_prestacao)
    tabela['valor_presente_prestacao'].append(vpresente(valor_da_prestacao, taxa_dia, dias_acumulados))
    tabela['iof_parcela'].append(iof_parcela)
    tabela['taxa_iof_total'].append(taxa_iof_dia * dias_iof)

    # print(f"{parcela=} {saldo_devedor=:10.2f} {amortizacao=:10.2f} {juros=:10.2f} {valor_da_prestacao=:10.2f}")


In [25]:
tabela_valor_solicidado_dado = pd.DataFrame(tabela)
tabela_valor_solicidado_dado.style.format({
    'dias_parcela': '{:d}',
    'dias_acumulados': '{:d}',
    'saldo_devedor': '{:.2f}',
    'amortizacao': '{:.2f}',
    'juros': '{:.2f}',
    'valor_prestacao': '{:.2f}',
    'valor_presente_prestacao': '{:.2f}'
})

,parcela,data_vencimento,dias_parcela,dias_acumulados,saldo_devedor,amortizacao,juros,valor_prestacao,valor_presente_prestacao,iof_parcela,taxa_iof_total
0,1,2021-02-05,31,31,18763.05,1236.95,206.70,1443.65,1428.88,7.844721,0.002542
1,2,2021-03-05,28,59,17494.47,1268.58,175.06,1443.65,1415.67,10.958035,0.004838
2,3,2021-04-05,31,90,16231.63,1262.84,180.81,1443.65,1401.19,14.118576,0.007380
3,4,2021-05-05,30,120,14950.29,1281.33,162.32,1443.65,1387.32,17.477371,0.009840
4,5,2021-06-05,31,151,13661.16,1289.14,154.51,1443.65,1373.13,20.860805,0.012382
5,6,2021-07-05,30,181,12354.12,1307.04,136.61,1443.65,1359.53,24.365781,0.014842
6,7,2021-08-05,31,212,11038.15,1315.97,127.68,1443.65,1345.62,27.877466,0.017384
7,8,2021-09-05,31,243,9708.58,1329.57,114.08,1443.65,1331.86,31.545344,0.019926
8,9,2021-10-05,30,273,8362.02,1346.56,97.09,1443.65,1318.67,35.261088,0.022386
9,10,2021-11-05,31,304,7004.79,1357.23,86.42,1443.65,1305.18,38.990404,0.024928


In [26]:
tabela_valor_solicidado_dado[['amortizacao', 'juros', 'valor_prestacao', 'valor_presente_prestacao', 'iof_parcela']].sum()

amortizacao                 20000.000000
juros                        1654.726522
valor_prestacao             21654.726522
valor_presente_prestacao    20000.000000
iof_parcela                   462.079614
dtype: float64

In [28]:
valor_financiado = valor_solicitado + tabela_valor_solicidado_dado['iof_parcela'].sum()
valor_financiado

np.float64(20462.079614463742)

In [41]:
valor_da_prestacao = valor_financiado / fator_de_financiamento

print(f"{valor_da_prestacao=:10.2f}")

valor_da_prestacao=   1477.00


In [42]:
saldo_devedor = valor_financiado

tabela = defaultdict(list)

for parcela in range(1, numero_de_prestacoes+1):

    dia_vencimento_parcela_atual = data_vencimento_parcelas[parcela - 1]
    if parcela == 1:
        dias_parcela = (dia_vencimento_parcela_atual - data_liberacao).days
    else:
        dia_vencimento_parcela_anterior = data_vencimento_parcelas[parcela - 2]
        dias_parcela = (dia_vencimento_parcela_atual - dia_vencimento_parcela_anterior).days
    dias_acumulados = (dia_vencimento_parcela_atual - data_liberacao).days

    juros = vfuturo(saldo_devedor, taxa_dia, dias_parcela) - saldo_devedor
    amortizacao = valor_da_prestacao - juros # principal

    saldo_devedor -= amortizacao

    tabela['parcela'].append(parcela)
    tabela['data_vencimento'].append(dia_vencimento_parcela_atual)
    tabela['dias_parcela'].append(dias_parcela)
    tabela['dias_acumulados'].append(dias_acumulados)
    tabela['saldo_devedor'].append(saldo_devedor)
    tabela['amortizacao'].append(amortizacao)
    tabela['juros'].append(juros)
    tabela['valor_prestacao'].append(valor_da_prestacao)
    tabela['valor_presente_prestacao'].append(vpresente(valor_da_prestacao, taxa_dia, dias_acumulados))

    # print(f"{parcela=} {saldo_devedor=:10.2f} {amortizacao=:10.2f} {juros=:10.2f} {valor_da_prestacao=:10.2f}")


In [43]:
tabela_valor_solicidado_dado = pd.DataFrame(tabela)
tabela_valor_solicidado_dado.style.format({
    'dias_parcela': '{:d}',
    'dias_acumulados': '{:d}',
    'saldo_devedor': '{:.2f}',
    'amortizacao': '{:.2f}',
    'juros': '{:.2f}',
    'valor_prestacao': '{:.2f}',
    'valor_presente_prestacao': '{:.2f}'
})

,parcela,data_vencimento,dias_parcela,dias_acumulados,saldo_devedor,amortizacao,juros,valor_prestacao,valor_presente_prestacao
0,1,2021-02-05,31,31,19196.55,1265.53,211.48,1477.00,1461.89
1,2,2021-03-05,28,59,17898.66,1297.89,179.11,1477.00,1448.38
2,3,2021-04-05,31,90,16606.64,1292.02,184.98,1477.00,1433.56
3,4,2021-05-05,30,120,15295.70,1310.94,166.07,1477.00,1419.37
4,5,2021-06-05,31,151,13976.78,1318.92,158.08,1477.00,1404.85
5,6,2021-07-05,30,181,12639.55,1337.23,139.77,1477.00,1390.94
6,7,2021-08-05,31,212,11293.18,1346.37,130.63,1477.00,1376.71
7,8,2021-09-05,31,243,9932.89,1360.29,116.72,1477.00,1362.63
8,9,2021-10-05,30,273,8555.22,1377.67,99.33,1477.00,1349.14
9,10,2021-11-05,31,304,7166.63,1388.58,88.42,1477.00,1335.34


In [44]:
tabela_valor_solicidado_dado[['amortizacao', 'juros', 'valor_prestacao', 'valor_presente_prestacao']].sum()

amortizacao                 20462.079614
juros                        1692.957292
valor_prestacao             22155.036906
valor_presente_prestacao    20462.079614
dtype: float64